In [88]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern1 = [{"LOWER": "go"},  {"LOWER": "to"}]
matcher.add("GOTO", [pattern1])
pattern2 = [{"LOWER": "column"}]
matcher.add("COLUMN", [pattern2])
pattern3 = [{"LOWER" : "row"}]
matcher.add("ROW", [pattern3])
pattern4 = [{"LOWER" : "enter"}]
matcher.add("ENTER" , [pattern4])

In [3]:
from openpyxl import Workbook, load_workbook
wb = load_workbook('new_Book1.xlsx')
ws = wb.active
print(ws)

<Worksheet "Sheet1">


In [4]:
ws['A1'].value = 'Hello'

In [90]:
def enter_sheet(row,column,inp):
    ws[column+row].value = inp 

In [91]:
digit_mapping = {'one' : 1,

'two' : 2,
'three' : 3,
'four' : 4,
'five' : 5

} 

In [92]:
import azure.cognitiveservices.speech as speechsdk
import time

'''

def recognize_from_microphone():
    speech_config = speechsdk.SpeechConfig(
                                           subscription = "5fd1c56ff99a400c8925c315c9ce56ec",
                                           region = "eastus"
                                           )
    speech_config.speech_recognition_language="en-US"

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("Speak into your microphone.")
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(speech_recognition_result.text))
        return speech_recognition_result.text
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")


'''
def recognize_from_microphone():
    speech_config = speechsdk.SpeechConfig(
                                           subscription = "5fd1c56ff99a400c8925c315c9ce56ec",
                                           region = "eastus"
                                           )
    speech_config.speech_recognition_language="en-US"
    speech_config.enable_dictation()

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    def stop_cb(evt):
        """callback that stops continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    print("Speak into your microphone.")
    #speech_recognition_result = speech_recognizer.recognize_once_async().get()
    # Start continuous speech recognition
    speech_recognition_result = speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)


    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(speech_recognition_result.text))
        return speech_recognition_result.text
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

In [93]:
#input = recognize_from_microphone()

In [94]:
def get_ls_matches(input):
    global doc
    doc = nlp(input)
    ls_matches1 = []
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        #print(match_id, string_id, start, end, span.text)
        ls_matches1.append((string_id,start,end))
    return ls_matches1

In [95]:
def place_data(input):
    column_text = {}
    ls_matches = get_ls_matches(input)
    for i,command in enumerate(ls_matches):
        if command[0] == 'ROW':
            j = command[2]
            #r = input.split()[j].replace(',','')
            r = doc[j].text
            print('Row : ',r)
        if command[0] == 'COLUMN':
            j = command[2]
            #c =  input.split()[j].replace(',','')
            c = doc[j].text
            print('column : ',c)

            flag = False
            if ls_matches[i+1][0] == 'ENTER':
                j = ls_matches[i+1][2]
                #print(j)
                try:
                    if ls_matches[i+2][0] == "GOTO":
                        flag = True
                        k = ls_matches[i+2][1]
                except:
                    pass
                if flag:
                    body = doc[j:k].text
                else:
                    body = doc[j:].text
                column_text[c] = body
                print('body : ', body)
    
    return {'row':r, 'column':column_text}

In [96]:
inp = "Go to row five, go to column B, enter wiry"
place_data(inp)

Row :  five
column :  B
body :  wiry


{'row': 'five', 'column': {'B': 'wiry'}}

In [97]:
place_loc = place_data(inp)

Row :  five
column :  B
body :  wiry


In [98]:
place_loc['column']

{'B': 'wiry'}

In [99]:
row = digit_mapping[place_loc['row']]
row = str(row)
for k,v in place_loc['column'].items():
    enter_sheet(row, k, v)

In [100]:
wb.save('new_Book1.xlsx')

In [3]:
import requests

resp = requests.post(' https://geography-sizing.azurewebsites.net/landing_page')
print(resp.text)
print(resp.status_code)

{"result":[{"company_name":"Adaptimmune","company_website":"https://www.adaptimmune.com","id":14},{"company_name":"ANGLE plc","company_website":"https://angleplc.com/","id":35},{"company_name":"GSK","company_website":"https://www.gsk.com","id":161},{"company_name":"Emergent BioSolutions Inc.","company_website":"https://www.emergentbiosolutions.com","id":672},{"company_name":"Simulations Plus, Inc.","company_website":"https://www.simulations-plus.com","id":708},{"company_name":"Boehringer Ingelheim Pharmaceuticals, Inc.","company_website":"https://www.boehringer-ingelheim.com","id":85},{"company_name":"ABS Inc.","company_website":"https://www.absbio.com","id":7},{"company_name":"ACROBiosystems","company_website":"https://www.acrobiosystems.com","id":11},{"company_name":"3DHISTECH","company_website":"https://www.3dhistech.com","id":2},{"company_name":"AACR Publications","company_website":"https://www.aacr.org","id":3},{"company_name":"AACRcentral","company_website":"https://www.aacr.org"

In [2]:
import requests
resp = requests.post(' https://geography-sizing.azurewebsites.net/geography_sizing',
json ={
	"id" : 14,
	"company_name" : "Adaptimmune",
	"company_website" : "https://www.adaptimmune.com",

})
resp.text

'{"Error":"Sales data not available"}\n'

In [2]:
import speech_recognition as sr

In [3]:
sr.Microphone.list_microphone_names()

AttributeError: Could not find PyAudio; check installation